# MSDS 453 Sentence based transformer chatbot

In [30]:
import pandas as pd
import numpy as np
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from collections import Counter
from dataclasses import dataclass
from timeit import default_timer as timer
import random
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import gensim
from gensim.models import Word2Vec

import spacy
from spacy import displacy

from spacy.matcher import Matcher
from spacy.tokens import Span

import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


from IPython.display import display, HTML

from typing import List, Callable, Dict, Tuple, Set

pd.set_option('max_colwidth', 600)
pd.set_option('display.max_rows', 500)

In [31]:
#Load Sentence Transformer model optimized for  sentence cosine similarity calculations

#The models below fully downloaded in Google Colab. This is the version of the google colab notebook but
#it was open in anoconda to be saved as pdf and the download graphics did not transfer properly so
#it seems like it didn't download. However, it did in the orignal google colab notebook, where all the
#analysis was run.

# model1 = 'multi-qa-MiniLM-L6-cos-v1'
# model2 = 'all-MiniLM-L12-v2'

# model = SentenceTransformer(model1)

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [33]:
# Only run this once, they will be downloaded.
nltk.download('stopwords',quiet=True)
nltk.download('wordnet',quiet=True)
nltk.download('punkt',quiet=True)
nltk.download('omw-1.4',quiet=True)


# # initializes an instance of NLTK's WordNet Lemmatizer
lemmer = nltk.stem.WordNetLemmatizer()
# # takes a list of tokens (words) as input and returns a list of their lemmas

def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]


# # creates a dictionary that maps the Unicode code point of each punctuation character to None
# # used to remove punctuation from text


remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)


def LemNormalize(text):
# creates a set of stop words & punctuation characters
  stop = set(stopwords.words('english') + list(string.punctuation))
  # tokenizes the input text into words, converts them to lowercase, and removes the stop words and punctuation
  # remaining words are joined back into a single string
  text2= ' '.join([i for i in word_tokenize(text.lower()) if i not in stop])
  # string is then converted to lowercase, the punctuation is removed
  return LemTokens(nltk.word_tokenize(text2.lower().translate(remove_punct_dict)))

In [34]:
#read in data
import pandas as pd

CORPUS_PATH = '/content/gdrive/MyDrive/Data/student_loans-3.txt'
# CORPUS_PATH = '/content/gdrive/MyDrive/Data/student_loans-2.xlsx'
f=open(CORPUS_PATH,'r',errors = 'ignore')
# df = pd.read_excel(CORPUS_PATH)

raw=f.read()
raw=raw.lower()# converts to lowercase

#create list of sentences and words
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences
word_tokens = nltk.word_tokenize(raw)# converts to list of words


# from_xlsx = df['Text'].tolist()
# for i in range(len(from_xlsx)):
#   from_xlsx[i] = from_xlsx[i].replace("\n", " ")
# all_text = " ".join(from_xlsx)
# raw = all_text.lower()

# page_tokens = from_xlsx




In [35]:
#create greetings and greetings function

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey")
GREETING_RESPONSES = ["Hello"]


# Checking for greetings
def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [36]:
# Generating response function
# def response(user_response):
#     chatbot_response=''
#     sentence_encodings=model.encode(sent_tokens, convert_to_tensor=True)# generate sentence transformer embeddings
#     sentence_encodings=sentence_encodings.cpu()
#     vals = cosine_similarity(sentence_encodings[-1].reshape(1, -1), sentence_encodings) #the chatbot conversation code
#     #in the next cell adds the question as the last sentence of the sentence tokens, before calling this response function.
#     #The code takes the last sentence (which is the question) and gets cosine similarities vs all the sentences in the corpus,
#     #including itself
#     idx=vals.argsort()[0][-2] #gets the index of the second highest similarity (the first highest would be the question itself)
#     flat = vals.flatten()#reduces dimension of cosine similarity array to be able to sort
#     flat.sort() #sort the cosine similarity values
#     second_cos_sim_val = flat[-2] #get the second highest cosine similarity value.
#     if(second_cos_sim_val==0): #check the second highest cosine similarity value. If it's zero return the no match response,
#         #else return highest cosine similarity sentence.
#         chatbot_response=chatbot_response+"Sorry, I do not have an answer to your question in my database. For more information, please visit https://studentaid.gov."
#         return chatbot_response
#     else:
#         chatbot_response = chatbot_response+sent_tokens[idx] #use index of highest cosine similarity to get original sentence
#         return chatbot_response

def response(user_response):
  robo_response=''
  #initializetoemptyresponse
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize)
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx=vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if(req_tfidf==0):
    robo_response = robo_response+"I am sorry! I don't understand you"
    return robo_response
  else:
    robo_response = robo_response + sent_tokens[idx]
    return robo_response



    # TfidfVec2 = TfidfVectorizer(tokenizer=LemNormalize)
    # specific_sent_tokens2 = nltk.sent_tokenize(page_tokens[idx])
    # specific_sent_tokens2.append(user_response)
    # tfidf2 = TfidfVec2.fit_transform(specific_sent_tokens2)
    # vals2 = cosine_similarity(tfidf2[-1], tfidf2)
    # idx2=vals2.argsort()[0][-2]
    # flat2 = vals2.flatten()
    # flat2.sort()
    # req_tfidf2 = flat2[-2]
    # if(req_tfidf2==0):
    #   robo_response=robo_response+"I am sorry! I don't understand you"
    #   return robo_response
    # robo_response = robo_response+specific_sent_tokens2[idx2]








In [37]:
#Chatbot interaction code

flag=True
print("Hi, my name is Finny! I aim to provide useful information regarding student loans and financial aid. To end session please type exit")
print("\n")

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if user_response!='exit':
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Answer: You are welcome - remember to consult your school counselor or visit https://studentaid.gov for more information.")
        else:
            if(greeting(user_response)!=None):
                print("Answer: "+greeting(user_response))
            else:
                sent_tokens.append(user_response)
                # page_tokens.append(user_response)
                word_tokens=word_tokens+nltk.word_tokenize(user_response)
                final_words=list(set(word_tokens))
                print("Answer: ",end="")
                print(response(user_response))
                print("\n")
                sent_tokens.remove(user_response)
                # page_tokens.remove(user_response)
    else:
        flag=False
        print("Goodbye and thank you for using the US Student Loans Information Chatbot. Please consult your school counselor or visit https://studentaid.gov for more information.")


Hi, my name is Finny! I aim to provide useful information regarding student loans and financial aid. To end session please type exit


what if I can't pay back my loans?
Answer: 

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


once aggregate limits are met, the student is ineligible for additional stafford loans until they pay back a portion of the borrowed funds.


what are the consequences of defaulting on my loans?
Answer: and the consequences are much more severe.


loan default consequences are?
Answer: and the consequences are much more severe.


consequences of default
Answer: and the consequences are much more severe.


thanks
Answer: You are welcome - remember to consult your school counselor or visit https://studentaid.gov for more information.
